### Package initialization and Loading all experiment info

In [ ]:
%load_ext autoreload
%autoreload complete

import pyeCAP as pyCAP
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import matplotlib.cm as cm
import sys
import os
import itertools
import scipy.io
import warnings
warnings.filterwarnings('ignore')

from Setting import ExperimentSettings
from plot_ecap import *
from Code_Blocks import *
from load_exp_info_csv import * 

setting = ExperimentSettings()
setting.file_location = r"H:\Other computers\My Computer (2)\H\eCAPData"
setting.file_, setting.Dir, setting.Status, setting.Date, setting.Path, setting.df_exp_info = load_exp_info_csv(os.path.join(setting.file_location,"Collection_of_experimental_file_info.csv"),file_location)
pd.set_option('display.max_rows', None)
display(df_exp_info.iloc[:, :5])

### Functions for data preprocessing, computing traces, and plotting traces.

In [ ]:
# import pandas as pd
# import csv

# def save_parameters_to_csv():
    # Create a dictionary of parameters and their values
    # parameters = {
    #     'Experiment settings': {
    #         'ExpNo': ExpNo,
    #         'filelist1': filelist1,
    #         'par_list1_': par_list1_,
    #         'stores': stores
    #     },
    #     'Data processing parameters': {
    #         'Pulse_Ind': Pulse_Ind,
    #         'ch_eCAP': ch_eCAP,
    #         'common_ref': common_ref,
    #         'pl_on': pl_on,
    #         'lp_on': lp_on,
    #         'hp_on': hp_on,
    #         'filter_hp_type': filter_hp_type,
    #         'filter_lp_type': filter_lp_type,
    #         'sample_delay': sample_delay,
    #         'hp_ks': hp_ks,
    #         'hpfo': hpfo,
    #         'lpff': lpff,
    #         'flip_polarity': flip_polarity,
    #         'recording_type': recording_type,
    #         'Subtract_SA': Subtract_SA,
    #         'SA_ch': SA_ch,
    #         'use_median': use_median

    #     },
    #     'Figure parameters': {
    #         'name_of_figure': name_of_figure,
    #         'custom_titles': custom_titles,
    #         'Subtitle': Subtitle,
    #         'use_status': use_status,
    #         'figsz': figsz,
    #         'lgfnt': lgfnt,
    #         'lc': lc,
    #         'ax_sub': ax_sub,
    #         'PlotAggrePara': PlotAggrePara,
    #         'PlotAggreConds': PlotAggreConds,
    #         'autoclosefigures': autoclosefigures,
    #         'plot_individual_traces': plot_individual_traces,
    #         'trace_alpha': trace_alpha,
    #         'use_broken_axes': use_broken_axes,
    #         'broken_axes_ylims': broken_axes_ylims,
    #         'save_figure': save_figure,
    #         'file_format': file_format
    #     },
    #     'eCAP plot settings': {
    #         'eCAPspacing': eCAPspacing,
    #         'auto_ylim': auto_ylim,
    #         'y_lim_range': y_lim_range,
    #         't_ylim': t_ylim,
    #         't_xlim': t_xlim
    #     }
    # }

    # csv_file_name = f'{name_of_figure}_param.csv'

    # with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    #     csv_writer = csv.writer(csvfile)
    #     row_offset = 0

    #     # Save general parameters
    #     for group_name, group_params in parameters.items():
    #         csv_writer.writerow([group_name])

    #         for param_name, param_value in group_params.items():
    #             csv_writer.writerow([param_name, param_value])

    #         # Add spacing between groups
    #         for _ in range(2):
    #             csv_writer.writerow([])
    #         row_offset += 2

    #     # Collect and save trace parameters
    #     trace_params_list = []
    #     for expNo, file_no in itertools.product([ExpNo], filelist1):
    #         stim_data = pyCAP.Stim(Dir[expNo-1, file_no])
    #         for par_ind in par_list1_:
    #             if (0, par_ind) in stim_data.parameters.index:
    #                 params = stim_data.parameters.loc[(0, par_ind), :].to_dict()
    #                 params['ExpNo'] = expNo
    #                 params['file_no'] = file_no
    #                 params['par_ind'] = par_ind
    #                 trace_params_list.append(params)
    #             else:
    #                 print(f"Index {par_ind} not found in stim_data.parameters.")

    #     # Write trace parameters to the CSV file
    #     csv_writer.writerow(['Trace Parameters'])
    #     csv_writer.writerow(['ExpNo', 'file_no', 'par_ind', 'onset time (s)', 'offset time (s)', 'period (ms)', 'frequency (Hz)', 'pulse count', 'pulse amplitude (μA)', 'duration (ms)', 'pulse duration (ms)', 'interphase delay (ms)', 'channel'])

    #     for trace_params in trace_params_list:
    #         csv_writer.writerow([trace_params[key] for key in ['ExpNo', 'file_no', 'par_ind', 'onset time (s)', 'offset time (s)', 'period (ms)', 'frequency (Hz)', 'pulse count', 'pulse amplitude (μA)', 'duration (ms)', 'pulse duration (ms)', 'interphase delay (ms)', 'channel']])

            


### [stable] Multiple Conditions/channels/parameters without shifting baselines

In [21]:

##########################################
# Experiment settings
setting.ExpNo                 = 15            # Experiment number
setting.filelist1             = [5]             # List of files to process; filelist can also call list of conditions 
setting.par_list1_            = [5]         # List of parameter indices to plot
setting.stores                = 'CAPc'          # Data store

# Data processing parameters
setting.Pulse_Ind             = range(1, 299)   # Indices of pulses to process and average
setting.ch_eCAP               = [1,2,3,4,5]     # eCAP channels to process
setting.common_ref            = 0               # Common reference channel
setting.pl_on                 = 0               # powerline filter
setting.lp_on                 = 0               # Low-pass filter
setting.hp_on                 = 0               # High-pass filter
setting.filter_hp_type        = 'median'        # High-pass filter type
setting.filter_lp_type        = 'gaussian'      # Low-pass filter type
setting.sample_delay          = 24              # Sample delay
setting.hp_ks                 = 750             # High-pass median filter kernel size
setting.hpfo                  = 1               # High-pass filter order
setting.lpff                  = 5000            # Low-pass filter frequency
setting.flip_polarity         = False           # Flip polarity of the signal
setting.recording_type        = 'monopolar'     # Recording type (e.g. 'monopolar', 'bipolar', 'tripolar')
setting.Subtract_SA           = False           # Subtract Stimulus Artifact using rescaled empty channel
setting.SA_ch                 = 6               # empty channel for Stimulus Artifact
setting.use_median            = True            # use median as ensemble averaging of traces

# Figure parameters
setting.name_of_figure        = 'Poster_Fig1_emg'              # Name of the figure
setting.custom_titles         = None #["Cond. 1", "Cond. 2", "Cond. 3", "Cond. 4", "Cond. 5", "Cond. 6", "Cond. 7"] # Custom titles for subplots, if not use enter None 
setting.Subtitle              = ''              # Subtitle for the figure
setting.use_status            = True            # Use status in the title
setting.figsz                 = (7.5, 3.5)         # Figure size
setting.show_legend           = False           # Show legend
setting.lgfnt                 = 12              # Legend font size
setting.lc                    = 'upper right'   # Legend location
setting.ax_sub                = None            # Subplot axis (useful for multi-panel figures)
setting.PlotAggrePara         = 0               # Aggregate parameters in the plot
setting.PlotAggreConds        = 0               # Aggregate conditions in the plot
setting.autoclosefigures      = True            # Auto close previous figures
setting.plot_individual_traces= False            # Plot individual traces
setting.trace_alpha           = 1               # Individual trace transparency = trace_alpha/num_traces
setting.use_broken_axes       = True            # (optional) plot with broken axes  
setting.broken_axes_ylims     =  ((-550, -500),(-250, 200), (1900, 1950))  #Set use_broken_axes to True and provide custom ylims when calling plot_multi_cond_para_chs
setting.save_figure           = True             # Save the figure
setting.file_format           = 'png'            # File format (e.g., 'pdf', 'eps', 'png')

# eCAP plot settings
setting.eCAPspacing           = 500             # eCAP trace spacing
setting.auto_ylim             = False           # Auto Y-axis limits
setting.y_lim_range           = [-175, 75]      # Y-axis limits
setting.t_ylim                = [2, 20]         # Time range for Y-axis limits calculation
setting.t_xlim                = [0, 2]        # Time range for X-axis
##########################################
if setting.autoclosefigures:
    plt.close('all')
##########################################

# from brokenaxes import brokenaxes
# ax_sub = brokenaxes(ylims=broken_axes_ylims)

# Load stim data
stim_data          = pyCAP.Stim(Dir[setting.ExpNo - 1, setting.filelist1[-1]])
display(stim_data.parameters)

# plot and save plot_parameters
%matplotlib
plot_multi_cond_para_chs(setting)
# save_parameters_to_csv()

# plot and save each figure for each condition


onset time (s)  offset time (s)  period (ms)  frequency (Hz)  \
0 0        54.200566        66.199797    39.997437       25.001602   
  1        84.200571        96.199802    39.997437       25.001602   
  2       114.200576       126.199807    39.997437       25.001602   
  3       144.200581       156.199812    39.997437       25.001602   
  4       174.200586       186.199817    39.997437       25.001602   
  5       204.200591       216.199822    39.997437       25.001602   
  6       234.200596       246.199827    39.997437       25.001602   
  7       264.200602       276.199833    39.997437       25.001602   
  8       294.200607       306.199838    39.997437       25.001602   
  9       324.200612       336.199843    39.997437       25.001602   
  10      354.200617       366.199848    39.997437       25.001602   
  11      384.200622       396.199853    39.997437       25.001602   
  12      414.200627       426.199858    39.997437       25.001602   
  13      444.200632       456.199863    39.997437       25.001602   
  14      474.200637       486.199868    39.997437       25.001602   
  15      504.200643       516.199874    39.997437       25.001602   
  16      534.200648       546.199879    39.997437       25.001602   
  17      564.200653       576.199884    39.997437       25.001602   
  18      594.200658       606.199889    39.997437       25.001602   
  19      624.200663       636.199894    39.997437       25.001602   
  20      654.200668       666.199899    39.997437       25.001602   
  21      684.200673       696.199904    39.997437       25.001602   
  22      714.200678       726.199909    39.997437       25.001602   
  23      744.200684       756.199914    39.997437       25.001602   
  24      774.200689       786.199920    39.997437       25.001602   
  25      804.200694       816.199925    39.997437       25.001602   
  26      834.200699       846.199930    39.997437       25.001602   
  27      864.200704       876.199935    39.997437       25.001602   
  28      894.200709       906.199940    39.997437       25.001602   

      pulse count  pulse amplitude (μA)  duration (ms)  pulse duration (ms)  \
0 0           300                   0.0   11999.230957                 0.04   
  1           300                  10.0   11999.230957                 0.04   
  2           300                 -10.0   11999.230957                 0.04   
  3           300                  20.0   11999.230957                 0.04   
  4           300                 -20.0   11999.230957                 0.04   
  5           300                  50.0   11999.230957                 0.04   
  6           300                 -50.0   11999.230957                 0.04   
  7           300                 100.0   11999.230957                 0.04   
  8           300                -100.0   11999.230957                 0.04   
  9           300                 200.0   11999.230957                 0.04   
  10          300                -200.0   11999.230957                 0.04   
  11          300                 500.0   11999.230957                 0.04   
  12          300                -500.0   11999.230957                 0.04   
  13          300                1000.0   11999.230957                 0.04   
  14          300               -1000.0   11999.230957                 0.04   
  15          300                2000.0   11999.230957                 0.04   
  16          300               -2000.0   11999.230957                 0.04   
  17          300                3000.0   11999.230957                 0.04   
  18          300               -3000.0   11999.230957                 0.04   
  19          300                4000.0   11999.230957                 0.04   
  20          300               -4000.0   11999.230957                 0.04   
  21          300                5000.0   11999.230957                 0.04   
  22          300               -5000.0   11999.230957                 0.04   
  23     

Using matplotlib backend: TkAgg
5


onset time (s)  offset time (s)  period (ms)  frequency (Hz)  \
0 0        54.200566        66.199797    39.997437       25.001602   
  1        84.200571        96.199802    39.997437       25.001602   
  2       114.200576       126.199807    39.997437       25.001602   
  3       144.200581       156.199812    39.997437       25.001602   
  4       174.200586       186.199817    39.997437       25.001602   
  5       204.200591       216.199822    39.997437       25.001602   
  6       234.200596       246.199827    39.997437       25.001602   
  7       264.200602       276.199833    39.997437       25.001602   
  8       294.200607       306.199838    39.997437       25.001602   
  9       324.200612       336.199843    39.997437       25.001602   
  10      354.200617       366.199848    39.997437       25.001602   
  11      384.200622       396.199853    39.997437       25.001602   
  12      414.200627       426.199858    39.997437       25.001602   
  13      444.200632       456.199863    39.997437       25.001602   
  14      474.200637       486.199868    39.997437       25.001602   
  15      504.200643       516.199874    39.997437       25.001602   
  16      534.200648       546.199879    39.997437       25.001602   
  17      564.200653       576.199884    39.997437       25.001602   
  18      594.200658       606.199889    39.997437       25.001602   
  19      624.200663       636.199894    39.997437       25.001602   
  20      654.200668       666.199899    39.997437       25.001602   
  21      684.200673       696.199904    39.997437       25.001602   
  22      714.200678       726.199909    39.997437       25.001602   
  23      744.200684       756.199914    39.997437       25.001602   
  24      774.200689       786.199920    39.997437       25.001602   
  25      804.200694       816.199925    39.997437       25.001602   
  26      834.200699       846.199930    39.997437       25.001602   
  27      864.200704       876.199935    39.997437       25.001602   
  28      894.200709       906.199940    39.997437       25.001602   

      pulse count  pulse amplitude (μA)  duration (ms)  pulse duration (ms)  \
0 0           300                   0.0   11999.230957                 0.04   
  1           300                  10.0   11999.230957                 0.04   
  2           300                 -10.0   11999.230957                 0.04   
  3           300                  20.0   11999.230957                 0.04   
  4           300                 -20.0   11999.230957                 0.04   
  5           300                  50.0   11999.230957                 0.04   
  6           300                 -50.0   11999.230957                 0.04   
  7           300                 100.0   11999.230957                 0.04   
  8           300                -100.0   11999.230957                 0.04   
  9           300                 200.0   11999.230957                 0.04   
  10          300                -200.0   11999.230957                 0.04   
  11          300                 500.0   11999.230957                 0.04   
  12          300                -500.0   11999.230957                 0.04   
  13          300                1000.0   11999.230957                 0.04   
  14          300               -1000.0   11999.230957                 0.04   
  15          300                2000.0   11999.230957                 0.04   
  16          300               -2000.0   11999.230957                 0.04   
  17          300                3000.0   11999.230957                 0.04   
  18          300               -3000.0   11999.230957                 0.04   
  19          300                4000.0   11999.230957                 0.04   
  20          300               -4000.0   11999.230957                 0.04   
  21          300                5000.0   11999.230957                 0.04   
  22          300               -5000.0   11999.230957                 0.04   
  23     

### save averaged traces to df -> .mat

In [18]:
##########################################
# Experiment settings
ExpNo                 = 13              # Experiment number
filelist1             = [0,1,2,3,4]     # 21,27,28,29,30,31] # List of files to process
par_list1_            = range(0,15)     # List of parameter indices to plot
stores                = 'CAPc'          # Data store

# Data processing parameters
Pulse_Ind             = range(0, 500)   # Indices of pulses to process
ch_eCAP               = [1,2,3,4,5]     # eCAP channels to process
common_ref            = 0               # Common reference channel
pl_on                 = 0               # powerline filter
lp_on                 = 0               # Low-pass filter
hp_on                 = 0               # High-pass filter
filter_hp_type        = 'median'        # High-pass filter type
filter_lp_type        = 'gaussian'      # Low-pass filter type
sample_delay          = 24              # Sample delay
N_Samples_to_Stim     = 10              # Number of samples to stim
hp_ks                 = 750             # High-pass median filter kernel size
hpfo                  = 1               # High-pass filter order
lpff                  = 5000            # Low-pass filter frequency
flip_polarity         = False           # Flip polarity of the signal
recording_type        = 'monopolar'     # Recording type (e.g. 'monopolar', 'bipolar', 'tripolar')
Subtract_SA           = False           # Subtract Stimulus Artifact using rescaled empty channel
SA_ch                 = 6               # empty channel for Stimulus Artifact
use_median            = True            # use median as ensemble averaging of traces

# Figure parameters
name_of_figure        = 'Poster_Fig1_emg'              # Name of the figure
custom_titles         = None #["Cond. 1", "Cond. 2", "Cond. 3", "Cond. 4", "Cond. 5", "Cond. 6", "Cond. 7"] # Custom titles for subplots, if not use enter None 
Subtitle              = ''              # Subtitle for the figure
use_status            = True            # Use status in the title
figsz                 = (7.5, 3.5)         # Figure size
show_legend           = False           # Show legend
lgfnt                 = 12              # Legend font size
lc                    = 'upper right'   # Legend location
ax_sub                = None            # Subplot axis (useful for multi-panel figures)
PlotAggrePara         = 1               # Aggregate parameters in the plot
PlotAggreConds        = 0               # Aggregate conditions in the plot
autoclosefigures      = True            # Auto close previous figures
plot_individual_traces= False            # Plot individual traces
trace_alpha           = 1               # Individual trace transparency = trace_alpha/num_traces
use_broken_axes       = True            # (optional) plot with broken axes  
broken_axes_ylims     =  ((-550, -500),(-250, 200), (1900, 1950))  #Set use_broken_axes to True and provide custom ylims when calling plot_multi_cond_para_chs
save_figure           = True             # Save the figure
file_format           = 'png'            # File format (e.g., 'pdf', 'eps', 'png')

# eCAP plot settings
eCAPspacing           = 500             # eCAP trace spacing
auto_ylim             = False           # Auto Y-axis limits
y_lim_range           = [-175, 75]      # Y-axis limits
t_ylim                = [2, 20]         # Time range for Y-axis limits calculation
t_xlim                = [0, 2]        # Time range for X-axis
##########################################
if autoclosefigures:
    plt.close('all')
##########################################

# Load stim data
stim_data          = pyCAP.Stim(Dir[ExpNo - 1, filelist1[-1]])
display(stim_data.parameters)
# plot and save plot_parameters
# plot_multi_cond_para_chs()
column_5=stim_data.parameters.loc[:, "pulse amplitude (μA)"]

onset time (s)  offset time (s)  period (ms)  frequency (Hz)  \
0 0        38.374420        58.373139    39.997437       25.001602   
  1        68.374426        88.373144    39.997437       25.001602   
  2        98.374431       118.373149    39.997437       25.001602   
  3       128.374436       148.373154    39.997437       25.001602   
  4       158.374441       178.373159    39.997437       25.001602   
  5       188.374446       208.373164    39.997437       25.001602   
  6       218.374451       238.373169    39.997437       25.001602   
  7       248.374456       268.373175    39.997437       25.001602   
  8       278.374461       298.373180    39.997437       25.001602   
  9       308.374467       328.373185    39.997437       25.001602   
  10      338.374472       358.373190    39.997437       25.001602   
  11      368.374477       388.373195    39.997437       25.001602   
  12      398.374482       418.373200    39.997437       25.001602   
  13      428.374487       448.373205    39.997437       25.001602   
  14      458.374492       478.373210    39.997437       25.001602   

      pulse count  pulse amplitude (μA)  duration (ms)  pulse duration (ms)  \
0 0           500                   0.0   19998.718262                 0.04   
  1           500                  10.0   19998.718262                 0.04   
  2           500                  20.0   19998.718262                 0.04   
  3           500                  50.0   19998.718262                 0.04   
  4           500                 100.0   19998.718262                 0.04   
  5           500                 200.0   19998.718262                 0.04   
  6           500                 500.0   19998.718262                 0.04   
  7           500                1000.0   19998.718262                 0.04   
  8           500                2000.0   19998.718262                 0.04   
  9           500                3000.0   19998.718262                 0.04   
  10          500                4000.0   19998.718262                 0.04   
  11          500                5000.0   19998.718262                 0.04   
  12          500                6000.0   19998.718262                 0.04   
  13          500                8000.0   19998.718262                 0.04   
  14          500               10000.0   19998.718262                 0.04   

      delay (ms)  channel  bipolar channel voice store  
0 0          0.0        1              2.0     A  eS1p  
  1          0.0        1              2.0     A  eS1p  
  2          0.0        1              2.0     A  eS1p  
  3          0.0        1              2.0     A  eS1p  
  4          0.0        1              2.0     A  eS1p  
  5          0.0        1              2.0     A  eS1p  
  6          0.0        1              2.0     A  eS1p  
  7          0.0        1              2.0     A  eS1p  
  8          0.0        1              2.0     A  eS1p  
  9          0.0        1              2.0     A  eS1p  
  10         0.0        1              2.0     A  eS1p  
  11         0.0        1              2.0     A  eS1p  
  12         0.0        1              2.0     A  eS1p  
  13         0.0        1              2.0     A  eS1p  
  14         0.0        1              2.0     A  eS1p

In [ ]:
# stim_data.parameters.shape
# column_index = 4  # Column indices start from 0, so 4 corresponds to the 5th column
# column_5 = stim_data.parameters.iloc[:, column_index]
# print(column_5)
import re

# num_columns = stim_data.parameters.shape[1]

# for column_index in range(num_columns):
for column_index, ignore in enumerate(stim_data.parameters):
    column_data = stim_data.parameters.iloc[:, column_index]
    # Remove parentheses
    column_data.name = re.sub(r'\(|\)', '', column_data.name)
    
    # Replace spaces with underscores
    column_data.name = column_data.name.replace(' ', '_')
    print(column_index)
    print(column_data.name)
    print(column_data.values)

In [ ]:
import scipy.io
import numpy as np
from scipy.io import savemat
import re

for file_Ind in filelist1:
    data_dict = {}  # Initialize a dictionary to collect all the data for this file

    for par_Ind in par_list1_:
        index_ar = (0, par_Ind)
        ecap_data, Amp, Pulses = preprocess_data(Dir, file_Ind, ExpNo, sample_delay=sample_delay)

        traces = compute_traces(ecap_data, index_ar, Pulse_Ind, ch_eCAP)

        time_vector = ecap_data.time(index_ar) * 1e3 - N_Samples_to_Stim * 1e3 / ecap_data.fs

        # Non-flipped portion
        # average the traces, convert to microvolts, and transpose the result
        averaged_trace_P = np.transpose(average_traces(traces, method='mean') * 1e6)
        # print(np.shape(averaged_trace_P))
        std_trace_P  = np.transpose(average_traces(traces, method='std') * 1e6)
        # print(np.shape(std_trace_P))
        
        # Add time_vector to the first column of the averaged_trace_P and std_trace_P 
        averaged_trace_P = np.column_stack((time_vector, averaged_trace_P))
        # print(np.shape(averaged_trace_P))
        std_trace_P = np.column_stack((time_vector, std_trace_P))
        # print(np.shape(std_trace_P))

        # Add the traces to the data dictionary
        data_dict[f'avg_trace_P_{par_Ind}'] = averaged_trace_P
        data_dict[f'std_trace_P_{par_Ind}'] = std_trace_P

        # Flipped portion
        # average the traces, convert to microvolts, and transpose the result
        averaged_trace_P = np.transpose(average_traces(traces, method='mean') * 1e6)
        # print(np.shape(averaged_trace_P))
        std_trace_P  = np.transpose(average_traces(traces, method='std') * 1e6)
        # print(np.shape(std_trace_P))
        # Reverse the channel order such that column 1 -> 5, 2->4 ....
        averaged_trace_P_rev = np.flip(averaged_trace_P[:, :], axis=1)
        # print(np.shape(averaged_trace_P_rev))
        std_trace_P_rev = np.flip(std_trace_P[:, :], axis=1)
        # print(np.shape(std_trace_P_rev))

        # Add time_vector to the first column of the reversed arrays
        averaged_trace_P_rev = np.column_stack((time_vector, averaged_trace_P_rev))
        # print(np.shape(averaged_trace_P_rev))
        std_trace_P_rev = np.column_stack((time_vector, std_trace_P_rev))
        # print(np.shape(std_trace_P_rev))

        # Add the reversed traces to the data dictionary
        data_dict[f'avg_trace_P_{par_Ind}_rev'] = averaged_trace_P_rev
        data_dict[f'std_trace_P_{par_Ind}_rev'] = std_trace_P_rev
    
    
    # for 
    # Storeing  the pulse amplitude data in the dictionary 
    # pulse_amplitude = stim_data.parameters['pulse amplitude (μA)'] #extracted the column of pulse amplitude from stim_data.parameters dataframe
    # pulse_amplitude_array = pulse_amplitude.values #converted into array 
    
    # data_dict['pulse_amplitude'] = pulse_amplitude_array

    # Storeing  the all stim data in the dictionary
    for column_index, ignore in enumerate(stim_data.parameters):
        column_data = stim_data.parameters.iloc[:, column_index]
        # Remove parentheses
        column_data.name = re.sub(r'\(|\)', '', column_data.name)
        # Replace     
        column_data.name = column_data.name.replace('μ', 'u')
        # Replace spaces with underscores
        column_data.name = column_data.name.replace(' ', '_')
        print(column_index)
        print(column_data.name)
        print(column_data.values)
        data_dict[column_data.name] = column_data.values

    
    file_name = f"ECAP_Data_ExpNo_{ExpNo}_File_{filelist1[file_Ind]}.mat"
    # Save to .mat file
    scipy.io.savemat(file_name, data_dict)







#     # Create unique filename
#     file_name = f"ECAP_Data_ExpNo_{ExpNo}_File_{file_Ind}.mat"
    

#     for idx, key in enumerate( averaged_trace_P_rev):
#        avg_traces = print(idx, key)


#     #saving the variable of pulse amplitude in the .mat file 
    
#     #scipy.io.savemat(file_name, all_dict)

# ##Trying something new by trasfering the pulse amplitude data to the desired file##
# source_file = 'ECAP_Data_ExpNo_13_File_0.mat'
# source_data = scipy.io.loadmat(source_file)

# # Extract the pulse_amplitude array from the source data
# pulse_amplitude = source_data['pulse_amplitude_dictionary']

# # Load the target .mat file
# target_file = 'ECAP_Data_ExpNo_13_File_0_Avg_test.mat'
# target_data = scipy.io.loadmat(target_file)

# # Insert the pulse_amplitude array into the target data
# target_data['pulse_amplitude_dictionary'] = pulse_amplitude

# # Save the modified target data to a new .mat file
# new_file = 'ECAP_Data_ExpNo_13_File_0_Modified.mat'
# scipy.io.savemat(new_file, target_data)


#     #pulse_amplitude_dict  = {'pulse_amplitude_dictionary': pulse_amplitude_transposed} #created a dictionary 
# print(pulse_amplitude_dict)
# savemat(filename, pulse_amplitude_dict, format='5', long_field_names=True)
#     #savemat(file_name, all_dict , format='5', long_field_names=True)

#         #11_july_code scipy.io.savemat(filename, pulse_amplitude_dict)    




In [ ]:
print(type(pulse_amplitude_transposed))

In [ ]:
pulse_amplitude_array = pulse_amplitude.values #converted into array 
print(type(pulse_amplitude_array))

In [ ]:
pulse_amplitude = stim_data.parameters['pulse amplitude (μA)']

print(type(pulse_amplitude))

In [ ]:
#for understanding how dictionary is getting stored in .mat file 
import scipy.io

#create a dictionary with sample data 
my_dict = {'name':'John', 'age':25 , 'city' : 'New York'}

#Specify the filename for the mat file
filename_xyz = 'example.mat'

#save the dictionary to a .mat file
scipy.io.savemat(filename_xyz,{'my_dict':my_dict}, format='5', long_field_names=True) 

In [ ]:
#for understanding the concept of scipy.to formatting
from scipy.io import savemat
import scipy.io
a = np.arange(20)
mdic = {"xyz" : a, "label" : "experiment"}
mdic
savemat("matlab_matrix.mat" , mdic)

In [ ]:
pulse_amplitude_dict

In [ ]:

#savemat() 
###  Bhoomika's code starts here
from scipy.io import loadmat,savemat
import numpy as np
import pdb
data = loadmat('ECAP_Data_ExpNo_13_File_0_Avg_example.mat') 
my_list_trace = []
#trace3 = traces[3,:]
#data['meanTrace'] = trace3
#trace4 = traces[4,:]
#trace5 = traces[5,:]
#trace6 = traces[6,:]
#trace7 = traces[7,:]
#trace8 = traces[8,:]
#trace9 = traces[9,:]
#trace10 = traces[10,:]
#my_list_trace.append(trace3)
#my_list_trace.append(trace4)
#my_list_trace.append(trace5)
#my_list_trace.append(trace6)
#my_list_trace.append(trace7)
#my_list_trace.append(trace8)
#my_list_trace.append(trace9)
#my_list_trace.append(trace10)
#traces_file_path = 'traces.mat'
#scipy.io.savemat(file_path, mdict={'traces': traces})
#scipy.io.savemat('my_list_trace')
arr_para = np.array([200, 500, 1000, 2000, 3000, 4000, 5000, 60000])
array_dict = {'array_parameters': arr_para} 
#pdb.set_trace()
#for idx, val in enumerate(data):
    #variables = print(idx, val)
#for idx1, val1 in enumerate(data1):
    #variables1 = print(idx1, val1)
print(type(arr_para ))
#savemat('array_parameters_values.mat', arr_para)
#savemat('file_x.mat',  variables )
#savemat('file_y.mat',  variables1 )
#dictionary = {'variables': variables, 'variables1': variables1}
#scipy.io.savemat('my_list_trace', 'array_dict ')
# shfits the time vector to the start of the stim pulse which is the n+1 th sample
n = 10
time_vector = time_vector - time_vector[n]   
## averaged traces operations
# # Reverse the row order of the averaged_traces array to address the figure composer's axis limitation
averaged_traces_reversed_rows = averaged_traces[::-1]

# Convert the reversed row array to a dataframe and transpose it
df_averaged_traces = pd.DataFrame(averaged_traces_reversed_rows).T

# Insert the time vector as the first column in the dataframe
df_averaged_traces.insert(0, 'Time', time_vector)

# Transpose the dataframe to have the time vector as the first row
df_averaged_traces = df_averaged_traces.T

# Save the transposed dataframe to a CSV file without row numbers (index)
# df_averaged_traces.to_csv('averaged_traces.csv', index=False)

# saved the non-transposed dataframe to a CSV file without row numbers (index)
df_averaged_traces = df_averaged_traces.T
# df_averaged_traces.to_csv('averaged_traces_not_transposed.csv', index=False)


## std traces operations
std_traces_reversed_rows = std_traces[::-1]

# Convert the reversed row array to a dataframe and transpose it
df_std_traces = pd.DataFrame(std_traces_reversed_rows).T

# Insert the time vector as the first column in the dataframe
df_std_traces.insert(0, 'Time', time_vector)

# Transpose the dataframe to have the time vector as the first row
df_std_traces = df_std_traces.T

# Save the transposed dataframe to a CSV file without row numbers (index)
# df_std_traces.to_csv('std_traces.csv', index=False)

# saved the non-transposed dataframe to a CSV file without row numbers (index)
df_std_traces = df_std_traces.T
# df_std_traces.to_csv('std_traces_not_transposed.csv', index=False)

filename = f"ECAP_Data_ExpNo_{ExpNo}_File_{filelist1[0]}.mat"
scipy.io.savemat(filename, df_averaged_traces)
pulse_amplitude = stim_data.parameters['pulse amplitude (μA)']
pulse_amplitude_dict  = {'pulse_amplitude_dictionary': pulse_amplitude}
sio.savemat('filename.mat', pulse_amplitude_dict) 

## averaged traces operations
# scipy.io.savemat('averaged_traces.mat', {'avg_traces': df_averaged_traces.values})
# scipy.io.savemat('averaged_traces_transposed.mat', {'avg_traces_T': df_averaged_traces.values})
# scipy.io.savemat('std_traces.mat', {'std_traces': df_std_traces.values})
# scipy.io.savemat('std_traces_transposed.mat', {'std_traces_T': df_std_traces.values})

In [ ]:
from scipy.io import loadmat,savemat
import numpy as np
import pdb
arr_para = np.array([200, 500, 1000, 2000, 3000, 4000, 5000, 6000]) #created an array of parameters
arr_dic = {'array_parameters': arr_para} #created a dictionary of the array
print(arr_dic)
scipy.io.savemat('array_parameters_values.mat', arr_dic) #saved the dictionary as a mat file
# data_avg_traces = scipy.io.loadmat('ECAP_Data_ExpNo_13_File_0_Avg_example.mat') #loading the average traces data
# Fetch the average trace variables
avg_traces = [
        data_avg_traces['avg_traces_P_0'],
        data_avg_traces['avg_traces_P_1'],
        data_avg_traces['avg_traces_P_2'],
        data_avg_traces['avg_traces_P_3'],
        data_avg_traces['avg_traces_P_4'],
        data_avg_traces['avg_traces_P_5'],
        data_avg_traces['avg_traces_P_6'],
        data_avg_traces['avg_traces_P_7'],
]
# Saving the average traces to separate .mat files
for i, trace in enumerate(avg_traces):
    # arr_para = arr_para[i]
    stim_amp = arr_para[i]
    file_name = f'avg_traces_{stim_amp}.mat'
    data_to_save = {'avg_traces': trace}
    scipy.io.savemat(file_name, data_to_save)

#row_index = 3
#trace3 = data['data_1'][row_index, :]
#print(trace3)

#my_array = np.empty((len(arr_para), len(trace1)))
#my_array[]


In [ ]:
import numpy as np
from scipy.io import savemat

# create a numpy array as example
arr = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# turn numpy array into a dictionary where the key is the name of the variable 
# in the mat file
arr_dict = {'my_array': arr}

# save it to a .mat file
savemat('my_file.mat', arr_dict)
